In [1]:
import os
import numpy as np
import cv2

#### 1. 读取output tensor

In [2]:
def txtToTensor(filepath):
    file = open(filepath, "r")
    datas = []

    while True:
        line =  file.readline()
        if not line:
            break
        temp_str = line.strip()

        if temp_str != "":
            data = float(temp_str)
            datas.append(data)


    result = np.asarray(datas)
    #result = np.expand_dims(result, -1)
    print(result.shape)

    #print(result.shape[-2:])

    #print(list(result.shape[-2:]))

    return result

#### 2. 后处理代码

In [3]:
def letterbox(im, new_shape=(640, 640), color=(0, 0, 0)):
    # Resize and pad image while meeting stride-multiple constraints
    shape = im.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])

    # Compute padding
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    #if shape[::-1] != new_unpad:  # resize
    #    im = cv2.resize(im, new_unpad, interpolation=cv2.INTER_LINEAR)
    #top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    #left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    #im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    #return im, ratio, (dw, dh)

In [4]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [5]:
def xywh2xyxy(x):
    # Convert [x, y, w, h] to [x1, y1, x2, y2]
    y = np.copy(x)
    y[:, 0] = x[:, 0] - x[:, 2] / 2  # top left x
    y[:, 1] = x[:, 1] - x[:, 3] / 2  # top left y
    y[:, 2] = x[:, 0] + x[:, 2] / 2  # bottom right x
    y[:, 3] = x[:, 1] + x[:, 3] / 2  # bottom right y
    return y

In [6]:
def process(input, mask, anchors):
    anchors = [anchors[i] for i in mask]
    grid_h, grid_w = map(int, input.shape[0:2])

    box_confidence = sigmoid(input[..., 4])
    box_confidence = np.expand_dims(box_confidence, axis=-1)

    box_class_probs = sigmoid(input[..., 5:])

    box_xy = sigmoid(input[..., :2]) * 2 - 0.5

    col = np.tile(np.arange(0, grid_w), grid_w).reshape(-1, grid_w)
    row = np.tile(np.arange(0, grid_h).reshape(-1, 1), grid_h)
    col = col.reshape(grid_h, grid_w, 1, 1).repeat(3, axis=-2)
    row = row.reshape(grid_h, grid_w, 1, 1).repeat(3, axis=-2)
    grid = np.concatenate((col, row), axis=-1)
    box_xy += grid
    box_xy *= int(IMG_SIZE / grid_h)

    box_wh = pow(sigmoid(input[..., 2:4]) * 2, 2)
    box_wh = box_wh * anchors

    box = np.concatenate((box_xy, box_wh), axis=-1)

    return box, box_confidence, box_class_probs




def filter_boxes(boxes, box_confidences, box_class_probs):
    """Filter boxes with box threshold. It's a bit different with origin yolov5 post process!

    # Arguments
        boxes: ndarray, boxes of objects.
        box_confidences: ndarray, confidences of objects.
        box_class_probs: ndarray, class_probs of objects.

    # Returns
        boxes: ndarray, filtered boxes.
        classes: ndarray, classes for boxes.
        scores: ndarray, scores for boxes.
    """
    box_classes = np.argmax(box_class_probs, axis=-1)
    box_class_scores = np.max(box_class_probs, axis=-1)
    pos = np.where(box_confidences[..., 0] >= BOX_THRESH)

    boxes = boxes[pos]
    classes = box_classes[pos]
    scores = box_class_scores[pos]

    return boxes, classes, scores


def nms_boxes(boxes, scores):
    """Suppress non-maximal boxes.

    # Arguments
        boxes: ndarray, boxes of objects.
        scores: ndarray, scores of objects.

    # Returns
        keep: ndarray, index of effective boxes.
    """
    x = boxes[:, 0]
    y = boxes[:, 1]
    w = boxes[:, 2] - boxes[:, 0]
    h = boxes[:, 3] - boxes[:, 1]

    areas = w * h
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)

        xx1 = np.maximum(x[i], x[order[1:]])
        yy1 = np.maximum(y[i], y[order[1:]])
        xx2 = np.minimum(x[i] + w[i], x[order[1:]] + w[order[1:]])
        yy2 = np.minimum(y[i] + h[i], y[order[1:]] + h[order[1:]])

        w1 = np.maximum(0.0, xx2 - xx1 + 0.00001)
        h1 = np.maximum(0.0, yy2 - yy1 + 0.00001)
        inter = w1 * h1

        ovr = inter / (areas[i] + areas[order[1:]] - inter)
        inds = np.where(ovr <= NMS_THRESH)[0]
        order = order[inds + 1]
    
    keep = np.array(keep)
    return keep


def yolov5_post_process(input_data):
    boxes, classes, scores = [], [], []
    for input, mask in zip(input_data, masks):
        b, c, s = process(input, mask, anchors)
        b, c, s = filter_boxes(b, c, s)
        boxes.append(b)
        classes.append(c)
        scores.append(s)

    boxes = np.concatenate(boxes)
    boxes = xywh2xyxy(boxes)
    classes = np.concatenate(classes)
    scores = np.concatenate(scores)

    nboxes, nclasses, nscores = [], [], []
    for c in set(classes):
        inds = np.where(classes == c)
        b = boxes[inds]
        c = classes[inds]
        s = scores[inds]

        keep = nms_boxes(b, s)

        nboxes.append(b[keep])
        nclasses.append(c[keep])
        nscores.append(s[keep])

    if not nclasses and not nscores:
        return None, None, None

    boxes = np.concatenate(nboxes)
    classes = np.concatenate(nclasses)
    scores = np.concatenate(nscores)

    return boxes, classes, scores


In [7]:
def draw(image, boxes, scores, classes):
    """Draw the boxes on the image.

    # Argument:
        image: original image.
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
        all_classes: all classes name.
    """
    for box, score, cl in zip(boxes, scores, classes):
        top, left, right, bottom = box
        print('class: {}, score: {}'.format(CLASSES[cl], score))
        print('box coordinate left,top,right,down: [{}, {}, {}, {}]'.format(top, left, right, bottom))
        top = int(top)
        left = int(left)
        right = int(right)
        bottom = int(bottom)

        #cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
        #cv2.putText(image, '{0} {1:.2f}'.format(CLASSES[cl], score),
        #            (top, left - 6),
        #            cv2.FONT_HERSHEY_SIMPLEX,
        #            0.6, (0, 0, 255), 2)

In [9]:

if __name__ == "__main__":
    #ONNX_MODEL = 'yolov5s.onnx'
    #MODEL_NAME = 'phone_rv1109'
    #RKNN_MODEL = f'./zoo/rknn/{MODEL_NAME}.rknn'
    # IMG_PATH = './phone'
    # SAVE_PATH = './save'
    BOX_THRESH = 0.2
    NMS_THRESH = 0.6
    #IMG_SIZE = 640
    IMG_SIZE = 320
    CLASSES = ("person", "phone")
    masks = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
    # 手机检测
    anchors = [[17, 21], [23, 36], [32, 57],
               [40, 98], [54, 136], [68, 189],
               [80, 122], [93, 249], [114, 167]]

    # 创建 RKNN 对象
    #rknn = RKNN()
    # 从当前目录加载 RKNN 模型 resnet_18
    #ret = rknn.load_rknn(path=RKNN_MODEL)
    #if ret != 0:
    #    print('Load Pytorch model failed!')
    #    exit(ret)
    # 初始化运行时环境
    #ret = rknn.init_runtime()
    #if ret != 0:
    #    print('Init runtime environment failed')
    #    exit(ret)

    IMG_PATH = "./20230703/"
    path_list = os.listdir(IMG_PATH)
    PLIST = [os.path.join(IMG_PATH, i) for i in path_list]
    #for i, IMG_PATH in enumerate(PLIST):
    for i in range(1):
        # Set inputs
        # img = cv2.imread(IMG_PATH)
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #img, ratio, (dw, dh) = letterbox(img)

        # Inference
        #print('--> Running model')
        #outputs = rknn.inference(inputs=[img])

        #input0_data = outputs[0]
        #input1_data = outputs[1]
        #input2_data = outputs[2]

        input0_data = txtToTensor("./20230703/0_outputTensor_000000000_00.txt")
        input0_data = input0_data.reshape(40, 3, 7, 40)
        input0_data = np.transpose(input0_data, (1, 2, 0, 3))

        input1_data = txtToTensor("./20230703/0_outputTensor_000000000_01.txt")
        input1_data = input1_data.reshape(10, 3, 7, 10)
        input1_data = np.transpose(input1_data, (1, 2, 0, 3))

        input2_data = txtToTensor("./20230703/0_outputTensor_000000000_02.txt")
        input2_data = input2_data.reshape(20, 3, 7, 20)
        input2_data = np.transpose(input2_data, (1, 2, 0, 3))

        '''
            output tensor dim: anchors * (5 + num_cls) * maps_h * maps_w
            while anchors=3, 5 means [offset_x, offset_y, offset_w, offset_h, object_confidence]
        '''
        input0_data = input0_data.reshape([3, -1] + list(input0_data.shape[-2:]))
        input1_data = input1_data.reshape([3, -1] + list(input1_data.shape[-2:]))
        input2_data = input2_data.reshape([3, -1] + list(input2_data.shape[-2:]))


        input_data = list()
        #input_data.append(np.transpose(input0_data, (2, 3, 0, 1)))
        #input_data.append(np.transpose(input1_data, (2, 3, 0, 1)))
        #input_data.append(np.transpose(input2_data, (2, 3, 0, 1)))

        # need change order, not sure right or not
        input_data.append(np.transpose(input1_data, (2, 3, 0, 1)))
        input_data.append(np.transpose(input2_data, (2, 3, 0, 1)))
        input_data.append(np.transpose(input0_data, (2, 3, 0, 1)))

        boxes, classes, scores = yolov5_post_process(input_data)

        print(boxes.shape)
        print(boxes)

        # img_1 = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        # if boxes is not None:
        #    draw(img_1, boxes, scores, classes)

        # cv2.imwrite(os.path.join(SAVE_PATH, path_list[i]), img_1)


(33600,)
(2100,)
(8400,)
(1192, 4)
[[-102.          -96.5         270.          152.5       ]
 [ 133.5         -56.5         226.5         192.5       ]
 [-152.          -63.5         304.          103.5       ]
 ...
 [ 159.05602906   -4.41648974  168.37657427   46.33589649]
 [-155.20158195  -15.61729359  149.24046177   33.77967683]
 [  -5.55635213   11.98408992  365.57404104   11.99017933]]
